In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import root_mean_squared_error

In [ ]:
data= pd.read_csv('C:/Users/visco/Downloads/playground-series-s6e1/train.csv')
tdata= pd.read_csv('C:/Users/visco/Downloads/playground-series-s6e1/test.csv')

data.columns
print(data.shape)


(630000, 13)


In [15]:
#Missing Value
data_hilang = data.isnull().sum()
print(data_hilang)

id                  0
age                 0
gender              0
course              0
study_hours         0
class_attendance    0
internet_access     0
sleep_hours         0
sleep_quality       0
study_method        0
facility_rating     0
exam_difficulty     0
exam_score          0
dtype: int64


In [16]:
y= data['exam_score']
X= data.drop(['id','exam_score'],axis=1)

In [17]:
for col in X:
    tipe= X[col].dtype
    print(f'{col} : {tipe}')

age : int64
gender : object
course : object
study_hours : float64
class_attendance : float64
internet_access : object
sleep_hours : float64
sleep_quality : object
study_method : object
facility_rating : object
exam_difficulty : object


In [18]:
# #feature engineering

#One-hot
X= pd.get_dummies(X,columns= ['facility_rating'])
X= pd.get_dummies(X,columns= ['sleep_quality'])
X= pd.get_dummies(X,columns= ['gender'])
X= pd.get_dummies(X, columns= ['internet_access'])
X= pd.get_dummies(X, columns= ['course'])
X= pd.get_dummies(X, columns= ['study_method'])
X= pd.get_dummies(X, columns= ['exam_difficulty'])

#Gabunhg
sleep_col= [col for col in X.columns if col.startswith('sleep_quality_')]
exam_col= [col for col in X.columns if col.startswith('exam_difficulty_')]
method_col= [col for col in X.columns if col.startswith(('study_method_'))]

#study method x exam difficulty
for i in exam_col:
    for j in method_col:
        X[f'{i}_{j}']= X[i]*X[j]

#sleep_hours x sleep_quality
for i in sleep_col:
    X[f'sleep_hours_{i}']= X['sleep_hours'] * X[i]

#study_hours x exam_difficulty
for i in exam_col:
    X[f'study_hours_{i}']= X['study_hours'] * X[i]

#study_hours x class attendance
X['rajin_value']= X['study_hours'] * X['class_attendance']

#sleep_hours x class_attendance
X['tipsen_value']= X['sleep_hours'] * X['class_attendance']
#


fitur_diskrit= X.dtypes == int

In [19]:
#mi score
# def make_mi_scores(X,y,discreate_feature):
#     mi_scores= mutual_info_regression(X,y,discrete_features=discreate_feature) #masukkan data feature importany
#     mi_scores= pd.Series(mi_scores, name= 'MI Scores', index= X.columns)
#     mi_scores= mi_scores.sort_values(ascending=False) #urutkan dari yang terbesar
#     return mi_scores

# mi_scores= make_mi_scores(X,y,fitur_diskrit)
# mi_scores[::]

In [20]:
#model
train_X,val_X,train_y,val_y= train_test_split(X,y, test_size=0.2,random_state=1)

model = RandomForestRegressor(n_estimators=300,max_depth=17,min_samples_leaf= 20,random_state=1)
model.fit(train_X,train_y)
val_pred= model.predict(val_X)
print('depth: 17, leaf: 30') 
print(root_mean_squared_error(val_y,val_pred))

    

    

depth: 17, leaf: 30
8.90494613249131


In [21]:
#feature importance
feature_importance = pd.Series(
    model.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

print(feature_importance)

rajin_value                                            0.797602
study_method_coaching                                  0.030388
tipsen_value                                           0.029950
sleep_hours_sleep_quality_poor                         0.020861
sleep_hours_sleep_quality_good                         0.017819
facility_rating_low                                    0.015885
study_hours                                            0.014007
sleep_quality_poor                                     0.013507
facility_rating_high                                   0.011099
class_attendance                                       0.009906
study_method_mixed                                     0.006935
sleep_hours                                            0.005791
facility_rating_medium                                 0.003607
study_hours_exam_difficulty_moderate                   0.003604
sleep_hours_sleep_quality_average                      0.003455
sleep_quality_good                      